In [21]:
!pip3 install -r ../requirements.txt

Requirement already up-to-date: ipywidgets in /usr/local/lib/python3.7/site-packages (7.5.1)


In [30]:
!git clone https://github.com/danibachar/idc_CANLab.git

Cloning into 'idc_CANLab'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 49 (delta 16), reused 44 (delta 13), pack-reused 0
Unpacking objects: 100% (49/49), done.
Updating files: 100% (17/17), done.


In [23]:
!pwd

/Users/danielbachar/Documents/IDC/idc_CANLab_New/notebooks
import _frozen_importlib # frozen
import _imp # builtin
import '_thread' # <class '_frozen_importlib.BuiltinImporter'>
import '_warnings' # <class '_frozen_importlib.BuiltinImporter'>
import '_weakref' # <class '_frozen_importlib.BuiltinImporter'>
# installing zipimport hook
import 'zipimport' # <class '_frozen_importlib.BuiltinImporter'>
# installed zipimport hook
import '_frozen_importlib_external' # <class '_frozen_importlib.FrozenImporter'>
import '_io' # <class '_frozen_importlib.BuiltinImporter'>
import 'marshal' # <class '_frozen_importlib.BuiltinImporter'>
import 'posix' # <class '_frozen_importlib.BuiltinImporter'>
import _thread # previously loaded ('_thread')
import '_thread' # <class '_frozen_importlib.BuiltinImporter'>
import _weakref # previously loaded ('_weakref')
import '_weakref' # <class '_frozen_importlib.BuiltinImporter'>
# /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.

# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud
# destroy google.cloud
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud
# destroy google.cloud
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud
# destroy google.cloud
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud/gapic
# destroy google.cloud.gapic
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud/gapic
# destroy google.cloud.gapic
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud/gapic/logging
# possible namespac

# destroy google.cloud.proto.devtools
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud/proto/devtools/clouderrorreporting
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google
# destroy google
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google
# destroy google
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google
# destroy google
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google
# destroy google
# possible namespace for /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/google/cloud
# destroy google.cloud
# possible namespace for /usr/lo

In [49]:
from idc_CANLab.functions import utils,plots,storage # Our functionalities

from ipywidgets import interact, interactive, fixed, interact_manual#, FileUpload
import ipywidgets as widgets
from IPython.display import display, FileLink

import os 
import io
import time
import json
# Data manipulation and calculation packages
import pandas as pd
import numpy as np
### Visualization
# Matplotlib
import matplotlib.pyplot as plt
# Bokeh
import bokeh.plotting
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
from bokeh.io import output_notebook
# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

# this will allow the notebook to reload/refresh automatically within the runtime
%reload_ext autoreload
%autoreload 2

Loading BokehJS ...

# Data Structures

dfs = {}
data_frame_names = []
cleaning_dfs = {}
cleaning_file_names = []

# Please enter your name, and a name for your experiment
## This will help us and you to build more meaningfull file names for your exported data and graphs

In [62]:
researcher_name = widgets.Text(
    value=utils.get_random_string(10),
    placeholder='Type your name'
)
project_name = widgets.Text(
    value=utils.get_random_string(6),
    placeholder='Type your project name'
)

names = widgets.VBox([researcher_name,project_name])
names

# File Uploads
## Please note we currently support only CSV files.
##    If your'e using Excel or other file formats please convert to CSV
###  Excel to CSV - https://knowledgebase.constantcontact.com/articles/KnowledgeBase/6409-saving-an-excel-file-as-a-csv-file?lang=en_US
### EDF to CSV - https://emotiv.gitbook.io/emotivpro/convert_edf_to_csv

# 1) Choose Primary file to upload - Only one file!

## If you alerady uploaded the file you can select it 

In [33]:
def choose_filename(filename):
  return filename
primary_file_name_dropdown = widgets.Dropdown(
    options=utils.list_all_files_in_dir("."),
    description='File name:',
)
# primary_file_name_dropdown
interact(choose_filename, filename=primary_file_name_dropdown)

interactive(children=(Dropdown(description='File name:', options=('./Outliers.ipynb', './Outlires_Lean.ipynb',…

<function __main__.choose_filename(filename)>

In [34]:
file_name = primary_file_name_dropdown.value
data_frame_names.append('primary')
dfs['primary'] = pd.read_csv(file_name)

In [42]:
_, url = plots.plots_by_group_and_features(dfs['primary'], "RECORDING_SESSION_LABEL", "AVERAGE_IA_1_SAMPLE_COUNT_%", "BIN_START_TIME", ["condition","load"])
# show(gridplot(utils.chunks(pls, 2)))


# output_file("test1.html",mode='inline')
# save(gridplot(utils.chunks(pls, 2)))



Groupping by -  RECORDING_SESSION_LABEL
y value by -  AVERAGE_IA_1_SAMPLE_COUNT_%
x value by -  BIN_START_TIME
feature matrix -  ['condition', 'load']
visit - to watch the graphs


'/Users/danielbachar/Documents/IDC/idc_CANLab_New/notebooks/test1.html'

In [53]:
print("visit - to watch the graphs",url)
print("https://google.com")

visit - to watch the graphs /Users/danielbachar/Documents/IDC/idc_CANLab_New/notebooks/test1.html
https://google.com


In [50]:
local_file = FileLink(url)
display(local_file)

/Users/danielbachar/Documents/IDC/idc_CANLab_New/notebooks/test1.html

In [52]:
v = '<img src="{}">'.format(url)
widgets.HTML(value=v)

HTML(value='<img src="/Users/danielbachar/Documents/IDC/idc_CANLab_New/notebooks/test1.html">')